In [1]:
import shutil

from core.gain import *
from core.rnn_predic import *
from core.models import *
from core.util import *
#from core.window import WindowGenerator, MissData, make_dataset_water, WaterDataGenerator
from core.window import WindowGenerator, make_dataset_gain, make_dataset_water
from core.file_open import make_dataframe
from core.miss_data import MissData
import json
%matplotlib widget
import os

In [2]:
def hour_to_day_mean(array):
    time = 24
    array = array.reshape((array.shape[0], array.shape[1] // time, time, array.shape[2]))
    array = array.mean(2)
    return array

In [3]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    width = input_width + label_width
    length = df.shape[0] - width
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    

    predictions = model(inputs).numpy()
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    plt.figure()
    input_index = np.array(range(0, length, 24))
    label_index = input_index + 24* (7 + predict_day)
    plt.plot(input_index, inputs_day[:, 0, :], label='input')
    plt.plot(label_index, label_day[:, predict_day, :], label='label')
    plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    plt.legend()
    #plt.savefig('test_plt.png')
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    #print("nse3 : ", nse3)
    #print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), pred_day, labels

In [4]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e) 


1 Physical GPUs, 1 Logical GPUs


In [5]:
data_path = 'data/'
parameters_dir = 'input'

parameters_file = 'input.json'
parameters_path = '{dir}/{file}'.format(dir=parameters_dir, file=parameters_file)

with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

gain_parameters = parameters['gain']
rnn_parameters = parameters['rnn']
file_parameters = parameters['file']

In [6]:
file_parameters['watershed']

'han_auto_13'

In [7]:
parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

In [8]:
pd.set_option('display.max_columns', 1000)

In [9]:
data_parameters = parameters['data']

interpolation_option = data_parameters['interpolation']
colum_idx = data_parameters['columns']
watershed = data_parameters['watershed']
file_names = data_parameters['files']
folder = data_parameters['directorys']
for i in range(len(folder)):
    folder[i] = watershed+folder[i]

In [10]:
__GAIN_TRAINING__ = gain_parameters['train']
gain_epochs = gain_parameters['max_epochs']
gain_in_setps = gain_parameters['input_width']
gain_out_setps = gain_parameters['label_width']
gain_batch_size = gain_parameters['batch_size']
gain_fill_no = gain_parameters['fill_width']
gain_shift = gain_parameters['shift_width']
gain_miss_rate = gain_parameters['miss_rate']

__RNN_TRAINING__ = rnn_parameters['train']
rnn_epochs = rnn_parameters['max_epochs']
rnn_in_setps = rnn_parameters['input_width']
rnn_out_steps = rnn_parameters['label_width']
rnn_batch_size = rnn_parameters['batch_size']
rnn_predict_day = rnn_parameters['predict_day']
rnn_target_column = rnn_parameters['target_column']

if rnn_predict_day < 3 or rnn_predict_day >5:
    print('predict_day err')
    exit(88)
rnn_predict_day -= 1

In [11]:
__GAIN_TRAINING__, gain_epochs ,__RNN_TRAINING__, rnn_epochs

(True, 2000, True, 20)

In [12]:
run_num = range(len(folder))
real_df_all = pd.DataFrame([])
target_all = target_mean = target_std = 0

gain_val_performance = {}
gain_performance = {}

length = len(run_num)

ddday = 31
mmmonth = 12

#length = 1

In [13]:
for i in range(length):

    idx = run_num[i]

    print('interpol flag : ', interpolation_option[idx])
    print('folder : ', data_path + folder[idx])
    print('colum_idx : ', colum_idx[idx])
    print('file_names[idx] : ', file_names[idx])

    #start = time.time()

    #if watershed == '한강_12days_test':
    #    df, times = make_dataframe_temp_12days(folder[idx], file_names[idx], colum_idx[idx], interpolate=interpolation_option[idx])
    #else:
    #df, times = make_dataframe(data_path+folder[idx], file_names[idx], colum_idx[idx], interpolation=interpolation_option[idx])
    df, times, mmmonth, ddday = make_dataframe(data_path+folder[idx], file_names[idx], 
                               colum_idx[idx], interpolation=interpolation_option[idx],
                              first_file_no=i, month=mmmonth, day=ddday)

    df_all, train_mean, train_std, df = normalize(df)


    if i == 0:
        dfff = df
        target_all = df_all
        target_std = train_std
        target_mean = train_mean
        start_year = str(times.iloc[0].year)
        end_year = str(times.iloc[-1].year)

    if interpolation_option[idx][0] == False:

        loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

        gain_calc_falg = True

        if __GAIN_TRAINING__ == True:
            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
            #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
        else:
            for file in loadfiles:
                if os.path.isfile('save/' + folder[idx]+file):
                    shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    #print('load file name : save/' + folder[idx]+file)
                else:
                    if file == 'miss.npy':
                        gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                        #print(folder[idx], ': is not miss.npy ', 'Miss date save : ', gain_calc_falg)

        if gain_calc_falg == True:
            #print('GainWindowGenerator in main')
            WindowGenerator.make_dataset = make_dataset_gain
            wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                          fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                          train_df = df_all, val_df = df_all, test_df = df_all, df = df)

            #gain = model_GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False, epochs=gain_epochs, training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')
            gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                              training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

            gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
            gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

            #print('file proc in main')
            if __GAIN_TRAINING__ == True:
                #dir = 'save/'+folder[i]
                if not os.path.exists('save/' + folder[idx]):
                    os.makedirs('save/'+folder[idx])
                for file in loadfiles:
                    shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

            #print('create_dataset_with_gain in main')
            #ori, gan = create_dataset_with_gain(gain=gain, window=wide_window, df=df)
            ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)
    else:
        gan = create_dataset_interpol(window=gain_in_setps, df=df)

    if i == 0 :
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.DataFrame(gan)
    else:
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)

interpol flag :  [False, False]
folder :  data/han/자동/
colum_idx :  :,[26,27,28,29,30,31,32,33]
file_names[idx] :  [['가평_2013.xlsx', '가평_2014.xlsx', '가평_2015.xlsx', '가평_2016.xlsx', '가평_2017.xlsx', '가평_2018.xlsx', '가평_2019.xlsx', '가평_2020.xlsx'], ['서상_2013.xlsx', '서상_2014.xlsx', '서상_2015.xlsx', '서상_2016.xlsx', '서상_2017.xlsx', '서상_2018.xlsx', '서상_2019.xlsx', '서상_2020.xlsx'], ['의암호_2013.xlsx', '의암호_2014.xlsx', '의암호_2015.xlsx', '의암호_2016.xlsx', '의암호_2017.xlsx', '의암호_2018.xlsx', '의암호_2019.xlsx', '의암호_2020.xlsx']]
data/han/자동/가평_2013.xlsx
data/han/자동/가평_2014.xlsx
data/han/자동/가평_2015.xlsx
data/han/자동/가평_2016.xlsx
data/han/자동/가평_2017.xlsx
data/han/자동/가평_2018.xlsx
data/han/자동/가평_2019.xlsx
data/han/자동/가평_2020.xlsx
time range in files :  2013-01-01 00:00  ~  2020-7-31 23:00
data/han/자동/서상_2013.xlsx
data/han/자동/서상_2014.xlsx
data/han/자동/서상_2015.xlsx
data/han/자동/서상_2016.xlsx
data/han/자동/서상_2017.xlsx
data/han/자동/서상_2018.xlsx
data/han/자동/서상_2019.xlsx
data/han/자동/서상_2020.xlsx
time range in files :  201

1/1 [==============================] - 0s 20ms/step - gen_loss: 12.2657 - disc_loss: 0.3673 - rmse: 0.4353 - val_loss: 0.4477
Epoch 50/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.8878 - disc_loss: 0.3510 - rmse: 0.3782 - val_loss: 0.3963
Epoch 51/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 12.0607 - disc_loss: 0.3445 - rmse: 0.4359 - val_loss: 0.5040
Epoch 52/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 12.8996 - disc_loss: 0.3738 - rmse: 0.4366 - val_loss: 0.4156
Epoch 53/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 14.3051 - disc_loss: 0.3648 - rmse: 0.4646 - val_loss: 0.3872
Epoch 54/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 12.4370 - disc_loss: 0.3584 - rmse: 0.4159 - val_loss: 0.4154
Epoch 55/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 11.1918 - disc_loss: 0.3596 - rmse: 0.3786 - val_loss: 0.4072
Epoch 56/2000
1/1 [================

1/1 [==============================] - 0s 20ms/step - gen_loss: 9.3634 - disc_loss: 0.3048 - rmse: 0.4677 - val_loss: 0.3979
Epoch 108/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.9909 - disc_loss: 0.2984 - rmse: 0.3939 - val_loss: 0.4353
Epoch 109/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.8207 - disc_loss: 0.2816 - rmse: 0.4528 - val_loss: 0.4240
Epoch 110/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.1216 - disc_loss: 0.2930 - rmse: 0.3173 - val_loss: 0.4107
Epoch 111/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 12.4367 - disc_loss: 0.2778 - rmse: 0.4711 - val_loss: 0.3759
Epoch 112/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.3427 - disc_loss: 0.2855 - rmse: 0.3409 - val_loss: 0.3868
Epoch 113/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 10.4665 - disc_loss: 0.2781 - rmse: 0.3804 - val_loss: 0.3553
Epoch 114/2000
1/1 [=============

1/1 [==============================] - 0s 20ms/step - gen_loss: 10.1280 - disc_loss: 0.2813 - rmse: 0.3838 - val_loss: 0.3424
Epoch 166/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 11.2642 - disc_loss: 0.2626 - rmse: 0.3932 - val_loss: 0.4536
Epoch 167/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.3357 - disc_loss: 0.2673 - rmse: 0.3737 - val_loss: 0.3525
Epoch 168/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.4012 - disc_loss: 0.2690 - rmse: 0.3765 - val_loss: 0.3401
Epoch 169/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.8425 - disc_loss: 0.2489 - rmse: 0.3073 - val_loss: 0.3531
Epoch 170/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.2395 - disc_loss: 0.2591 - rmse: 0.3443 - val_loss: 0.3294
Epoch 171/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.0339 - disc_loss: 0.2657 - rmse: 0.3170 - val_loss: 0.3807
Epoch 172/2000
1/1 [=============

1/1 [==============================] - 0s 20ms/step - gen_loss: 10.8177 - disc_loss: 0.2493 - rmse: 0.3836 - val_loss: 0.4170
Epoch 224/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 10.2974 - disc_loss: 0.2600 - rmse: 0.4019 - val_loss: 0.3580
Epoch 225/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.4214 - disc_loss: 0.2506 - rmse: 0.3373 - val_loss: 0.2798
Epoch 226/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.3459 - disc_loss: 0.2642 - rmse: 0.3866 - val_loss: 0.3301
Epoch 227/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.0960 - disc_loss: 0.2460 - rmse: 0.3157 - val_loss: 0.3867
Epoch 228/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.0091 - disc_loss: 0.2661 - rmse: 0.3393 - val_loss: 0.3762
Epoch 229/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.7414 - disc_loss: 0.2497 - rmse: 0.3693 - val_loss: 0.4225
Epoch 230/2000
1/1 [=============

...

1/1 [==============================] - 0s 20ms/step - gen_loss: 6.5961 - disc_loss: 0.1967 - rmse: 0.3545 - val_loss: 0.3722
Epoch 514/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 13.8568 - disc_loss: 0.2166 - rmse: 0.3273 - val_loss: 0.3545
Epoch 515/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.6992 - disc_loss: 0.2181 - rmse: 0.3622 - val_loss: 0.3592
Epoch 516/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.7972 - disc_loss: 0.2023 - rmse: 0.3171 - val_loss: 0.3392
Epoch 517/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.5703 - disc_loss: 0.2075 - rmse: 0.3959 - val_loss: 0.3180
Epoch 518/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.9404 - disc_loss: 0.1909 - rmse: 0.3626 - val_loss: 0.3490
Epoch 519/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.6312 - disc_loss: 0.2032 - rmse: 0.3841 - val_loss: 0.3616
Epoch 520/2000
1/1 [==============

1/1 [==============================] - 0s 20ms/step - gen_loss: 7.0196 - disc_loss: 0.2172 - rmse: 0.3920 - val_loss: 0.3395
Epoch 572/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.9340 - disc_loss: 0.1967 - rmse: 0.3072 - val_loss: 0.3354
Epoch 573/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.7743 - disc_loss: 0.2068 - rmse: 0.2948 - val_loss: 0.2953
Epoch 574/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.9624 - disc_loss: 0.2177 - rmse: 0.3768 - val_loss: 0.3219
Epoch 575/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.4176 - disc_loss: 0.2055 - rmse: 0.3439 - val_loss: 0.3031
Epoch 576/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 14.9374 - disc_loss: 0.1945 - rmse: 0.4090 - val_loss: 0.3386
Epoch 577/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 6.9335 - disc_loss: 0.2008 - rmse: 0.3692 - val_loss: 0.2842
Epoch 578/2000
1/1 [==============

1/1 [==============================] - 0s 20ms/step - gen_loss: 6.6865 - disc_loss: 0.1922 - rmse: 0.3355 - val_loss: 0.2897
Epoch 630/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.6786 - disc_loss: 0.1967 - rmse: 0.2718 - val_loss: 0.3268
Epoch 631/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.7505 - disc_loss: 0.2027 - rmse: 0.2738 - val_loss: 0.3120
Epoch 632/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.2901 - disc_loss: 0.1890 - rmse: 0.2941 - val_loss: 0.3298
Epoch 633/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.1885 - disc_loss: 0.1844 - rmse: 0.3125 - val_loss: 0.3647
Epoch 634/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.1422 - disc_loss: 0.1964 - rmse: 0.3670 - val_loss: 0.3319
Epoch 635/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.7011 - disc_loss: 0.1983 - rmse: 0.3509 - val_loss: 0.3185
Epoch 636/2000
1/1 [===============

1/1 [==============================] - 0s 20ms/step - gen_loss: 13.8861 - disc_loss: 0.1866 - rmse: 0.3227 - val_loss: 0.2921
Epoch 688/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.6763 - disc_loss: 0.1906 - rmse: 0.3032 - val_loss: 0.3283
Epoch 689/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.1741 - disc_loss: 0.1874 - rmse: 0.3626 - val_loss: 0.3696
Epoch 690/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.4211 - disc_loss: 0.1898 - rmse: 0.2875 - val_loss: 0.3402
Epoch 691/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.9948 - disc_loss: 0.2060 - rmse: 0.3251 - val_loss: 0.2946
Epoch 692/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.0752 - disc_loss: 0.1935 - rmse: 0.3131 - val_loss: 0.3043
Epoch 693/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.9370 - disc_loss: 0.1952 - rmse: 0.3411 - val_loss: 0.2934
Epoch 694/2000
1/1 [==============

1/1 [==============================] - 0s 20ms/step - gen_loss: 5.0996 - disc_loss: 0.1821 - rmse: 0.2735 - val_loss: 0.3550
Epoch 746/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.8880 - disc_loss: 0.1823 - rmse: 0.3023 - val_loss: 0.2932
Epoch 747/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.1276 - disc_loss: 0.1873 - rmse: 0.3533 - val_loss: 0.3398
Epoch 748/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.3902 - disc_loss: 0.1984 - rmse: 0.3242 - val_loss: 0.3054
Epoch 749/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 12.4880 - disc_loss: 0.1874 - rmse: 0.3205 - val_loss: 0.2986
Epoch 750/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.0964 - disc_loss: 0.1945 - rmse: 0.3281 - val_loss: 0.3587
Epoch 751/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.7174 - disc_loss: 0.1884 - rmse: 0.3090 - val_loss: 0.3543
Epoch 752/2000
1/1 [==============

In [14]:
#-----------------------------------------------------------------------------------------------------------------
#            RRRRRRRRRRRRRRRRRR          NNNNNN       NNNNNN      NNNNNN       NNNNNN
#           RRRRRRRRRRRRRRRRRRRR        NNNNNNN      NNNNNN      NNNNNNN      NNNNNN
#          RRRRRRR      RRRRRRRR       NNNNNNNN     NNNNNN      NNNNNNNN     NNNNNN
#         RRRRRRR      RRRRRRRR       NNNNNNNNN    NNNNNN      NNNNNNNNN    NNNNNN
#        RRRRRRRRRRRRRRRRRRRR        NNNNNNNNNN   NNNNNN      NNNNNNNNNN   NNNNNN
#       RRRRRRRRRRRRRRRRR           NNNNNNNNNNN  NNNNNN      NNNNNNNNNNN  NNNNNN 
#      RRRRRRR     RRRRRRR         NNNNNNN NNNN NNNNNN      NNNNNNN NNNN NNNNNN
#     RRRRRRR       RRRRRRR       NNNNNNN  NNNNNNNNNN      NNNNNNN  NNNNNNNNNN
#    RRRRRRR        RRRRRRR      NNNNNNN   NNNNNNNNN      NNNNNNN   NNNNNNNNN
#   RRRRRRR         RRRRRRR     NNNNNNN    NNNNNNNN      NNNNNNN    NNNNNNNN 
#  RRRRRRR         RRRRRRR     NNNNNNN     NNNNNNN      NNNNNNN     NNNNNNN
#-----------------------------------------------------------------------------------------------------------------

In [15]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}
target_col_idx :  6
out_num_features :  1


In [38]:
print(real_df_all.shape)
rnn_target_column = 'tp'
print(rnn_target_column)

#df_length = real_df_all.shape[0] - 3000
df_length = real_df_all.shape[0]

fake_df = real_df_all.iloc[:df_length,:]

(66456, 36)
tp


In [39]:
#train_df, val_df, test_df, test_df2 = dataset_slice(fake_df, 0.8, 0.1, 0.1)
#teg_check = 'train=8, val=1, test=1'

#train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.8, 0.1, 0.1)
#teg_check = 'train=8, test=1, val=1'

#val_df, test_df,train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'val=1, test=1, train=8'

#test_df, val_df, train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'test=1, val=1, train=8'

#val_df, train_df, test_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'val=1, train=8, test=1'

#test_df, train_df, val_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'test=1, train=8, val=1'

train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.7, 0.1, 0.2)
teg_check = 'train=7, test=1, val=2'

print('train_df.shape : ', train_df.shape, 'val_df.shape : ', val_df.shape, 'test_df.shape:' ,test_df.shape)

train_df.shape :  (46519, 36) val_df.shape :  (13292, 36) test_df.shape: (6645, 36)


In [40]:
#fake_df.shape[0]/8760

In [41]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}


In [42]:
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

target_col_idx :  6
out_num_features :  1


In [43]:
idx = [2, 4, 5, 6, 7]
pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]
indices = {name: i for i, name in enumerate(idx)}

In [44]:
plt.figure()
plt.plot(real_df_all.iloc[:, out_features[0]])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [45]:
plt.figure()
plt.plot(val_df.iloc[:, out_features[0]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [46]:
WindowGenerator.make_dataset = make_dataset_water
multi_window = WindowGenerator(
    input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
    out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
    train_df=train_df, val_df=val_df, test_df=test_df, test_df2=test_df2)

model_path = "save/" + watershed + "models/" + file_parameters['watershed'] + '/'+  pa[indices[target_col_idx]]
print("save model path : ", model_path)

if __RNN_TRAINING__:
    #if not os.path.exists('save/' + watershed):
    #    os.makedirs('save/' + watershed)
    if not os.path.exists(model_path):
        os.makedirs(model_path)

save model path :  save/han/models/han_auto_13/phosphorus/


In [47]:
#"save/" + watershed + "models/" + file_parameters['watershed'] + '/' + pa[indices[target_col_idx]]
#model_path

In [48]:
rnn_epochs, rnn_in_setps

(20, 240)

## 모델 학습

In [49]:
gru_model = model_gru(
    #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=15,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt")

Epoch 1/15
10/10 [==============================] - ETA: 0s - loss: 0.4082 - mean_absolute_error: 0.4034 - nse: -0.0428
Epoch 00001: val_nse improved from -inf to 0.24541, saving model to save/best_model.h5
10/10 [==============================] - 12s 1s/step - loss: 0.4082 - mean_absolute_error: 0.4034 - nse: -0.0428 - val_loss: 0.2613 - val_mean_absolute_error: 0.3672 - val_nse: 0.2454
Epoch 2/15
10/10 [==============================] - ETA: 0s - loss: 0.2097 - mean_absolute_error: 0.2764 - nse: 0.3574
Epoch 00002: val_nse improved from 0.24541 to 0.47395, saving model to save/best_model.h5
10/10 [==============================] - 12s 1s/step - loss: 0.2097 - mean_absolute_error: 0.2764 - nse: 0.3574 - val_loss: 0.1819 - val_mean_absolute_error: 0.3145 - val_nse: 0.4739
Epoch 3/15
10/10 [==============================] - ETA: 0s - loss: 0.1971 - mean_absolute_error: 0.2516 - nse: 0.4682
Epoch 00003: val_nse improved from 0.47395 to 0.47689, saving model to save/best_model.h5
10/10 [=

## core / window.py / 

In [50]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column , " ", out_features[0])
print("length : ",df_length)
print(teg_check)
print('nse_val : ', val_nse['GRU'])
#print("---------------")
#print(val_nse['GRU'])
#print(val_pbias['GRU'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 :  0.006129339
predict_day :  5
------------------------
0.0015799924 4.8851425e-06 0.002210236
save/han/models/han_auto_13/phosphorus/
year : 2013 ~ 2020
run :  range(0, 1)
target :  tp   6
length :  66456
train=7, test=1, val=2
nse_val :  [0.4119028]


In [51]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=real_df_all, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print('nse_all : ', val_nse['GRU'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 :  0.0061670537
predict_day :  5
------------------------
0.0012274378 3.918233e-06 0.0019794526
nse_all :  [0.46011972]


In [52]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=train_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)

print('nse_train : ', val_nse['GRU'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 :  0.006170021
predict_day :  5
------------------------
0.0010898376 3.5331561e-06 0.0018796691
nse_train :  [0.51134694]


In [53]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)

print('nse_test : ', val_nse['GRU'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 :  0.00627495
predict_day :  5
------------------------
0.0014471234 4.9860105e-06 0.0022329376
nse_test :  [0.2178489]


In [32]:
#train_df.shape, val_df.shape, test_df.shape

In [33]:
#-----------------------------------------------------------------------------------------------------------------
# 123SFACVTHYI78567UDBYUJM534523RDHGTHETYIE5I8I78O9P789J467UBDFGDASFSADCXVFDSWERTNERGRTTUTYAFGASDFSDBGFDF67E54563
# 64534FDSFVSDHT56SDFG656USRHSDYA3Y RTA WERGERGSDFGVTY34TASDF dfasdfghhgbcvbsdfgrwergfdsfvzxcvasdfaweqdqasdfderadf
# feasetttttttttttttttttttvxzcvxzvxcvxvnjfghjvb111nfghjdfhgdhthrtyyyyyyyyyyrtjliuioyuytrtyurtyughjuyityuityuityuijjty
# hgfgdrrtyetghfdghvcvbnhjfghjgggnfnmjhkuytlryewqwerasdfgghrthfgjyruhjyuihjknhybgtvfrcdezxcabnghrtyreeghghte45654y  
# tr345rgbbbdfghterty4534562345jhgy5462345234534n5345g345t3245d34f235345f345g34454f345t345er345234yt545y45tg45t534
#       SSSSSSSSSSSSSS         ASDFALKJIVAJLMSEFLKMALSKVJALKSRKLTGAKLSDGVLK   *****    ***************   
#            SSSSSSSSSSSSS               ******            ******              *****    *************       (*_*)  
#                   SSSSSSSS               ******            ******              *****    *****            ~( (>
#        SSSS         S  SSS               ******            ******              *****    *****            / (          
#      SSSS          S  SSSS               ******            ******              *****    *****                     
#     SSSSS            SSSS                ******             ******            ******    *****                     
#      SSSSSSSSSSSSSSSSSS                  ******              *********************      *****                   
#-----------------------------------------------------------------------------------------------------------------
stop = stop

NameError: name 'stop' is not defined

In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=real_df_all, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column, " ", out_features[0])
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [ ]:
from core.models import GRUModel, compile

In [ ]:
model112 = GRUModel(rnn_out_steps, out_num_features)
model112.load_weights(model_path+"gru.ckpt")
compile(model112)


In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    #model=model112,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    model=model112,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [ ]:
##.evaluate(multi_window.train)
#model112.evaluate(multi_window.val.repeat(-1), steps=100)

In [ ]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    
    #print(df.shape)
    #print(plot_col)
    
    width = input_width + label_width
    
    length = df.shape[0]
    length -= width
    
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    

    predictions = model(inputs).numpy()
    
    print("input : ", inputs.shape, "labels : ", labels.shape, "predictions : ", predictions.shape)
    #print(predictions.shape)
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    print("                label_day.shape : ",label_day.shape, "pred_day.shape : ", pred_day.shape)
    
    #plt.figure()
    #input_index = np.array(range(0, length, 24))
    #label_index = input_index + 24* (7 + predict_day)
    #plt.plot(input_index, inputs_day[:, 0, :], label='input')
    #plt.plot(label_index, label_day[:, predict_day, :], label='label')
    #plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    #plt.legend()
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    print("nse3 : ", nse3)
    print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), inputs_day, label_day, pred_day
    

In [ ]:

x = np.arange(len(val_nse))
width = 0.35
plt.figure()
plt.title(watershed + '  ['+start_year+','+end_year+']  ' + rnn_target_column)
plt.bar(x, val_pbias.values(), 0.3, label='PBIAS' )
plt.bar(x + width, val_nse.values(), 0.3, label='NSE')
plt.xticks(x,val_nse.keys(), rotation=0)
_ = plt.legend()
plt.show()
